# Lecture 12 : RNN

## Recurrent Neural Network

The output of a Layer is passed in as the next input.

RNN is usually used when a series of data is given. (Ex: Time Series Prediction, text genration, Speech recognition, Etc.)

## RNN Inside

input : x, output: h, model: A

A(x_t,h_t-1) = h_t

## Implementation : RNN in PyTorch

Example : Input Letters H,e,l,l,o 

### Input single letters 


In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable

# One-hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

# One Cell RNN input_dim (4) -> output_dim (2). sequence: 5
cell = nn.RNN(input_size=4, hidden_size=2, batch_first=True)

# (num_layers * num_directions, batch, hidden_size) whether batch_first=True or False
hidden = Variable(torch.randn(1, 1, 2))

# Propagate input through RNN
# Input: (batch, seq_len, input_size) when batch_first=True
inputs = Variable(torch.Tensor([h, e, l, l, o]))
for one in inputs:
    one = one.view(1, 1, -1)
    # Input: (batch, seq_len, input_size) when batch_first = True
    out, hidden = cell (one, hidden)
    print("one input size", one.size(), "out size", out.size())

one input size torch.Size([1, 1, 4]) out size torch.Size([1, 1, 2])
one input size torch.Size([1, 1, 4]) out size torch.Size([1, 1, 2])
one input size torch.Size([1, 1, 4]) out size torch.Size([1, 1, 2])
one input size torch.Size([1, 1, 4]) out size torch.Size([1, 1, 2])
one input size torch.Size([1, 1, 4]) out size torch.Size([1, 1, 2])


### Unfolding to N sequences

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

# One-hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

# One Cell RNN input_dim (4) -> output_dim (2). sequence: 5
cell = nn.RNN(input_size=4, hidden_size=2, batch_first=True)

# (num_layers * num_directions, batch, hidden_size) whether batch_first=True or False
hidden = Variable(torch.randn(1, 1, 2))

# Propagate input through RNN
# Input: (batch, seq_len, input_size) when batch_first=True
inputs = Variable(torch.Tensor([h, e, l, l, o]))


# We can do the whole at once
# Propagate input through RNN
# Input: (batch, seq_len, input_size) when batch_first=True
inputs = inputs.view(1, 5, -1)
out, hidden = cell(inputs, hidden)
print("sequence input size", inputs.size(), "out size", out.size())


sequence input size torch.Size([1, 5, 4]) out size torch.Size([1, 5, 2])


### Batching Input

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable

# One-hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

# One Cell RNN input_dim (4) -> output_dim (2). sequence: 5
cell = nn.RNN(input_size=4, hidden_size=2, batch_first=True)

# hidden : (num_layers * num_directions, batch, hidden_size) whether batch_first=True or False
hidden = Variable(torch.randn(1, 3, 2))


# One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
# 3 batches 'hello', 'eolll', 'lleel'
# rank = (3, 5, 4)
inputs = Variable(torch.Tensor([[h, e, l, l, o],
                                [e, o, l, l, l],
                                [l, l, e, e, l]]))

# Propagate input through RNN
# Input: (batch, seq_len, input_size) when batch_first=True
# B x S x I
out, hidden = cell(inputs, hidden)
print("sequence input size", inputs.size(), "out size", out.size())

# One Cell RNN input_dim (4) -> output_dim (2)
cell = nn.RNN(input_size=4, hidden_size=2)

# The given dimensions dim0 and dim1 are swapped.
inputs = inputs.transpose(dim0=0, dim1=1)
# Propagate input through RNN
# Input: (seq_len, batch_size, input_size) when batch_first=False (default)
# S x B x I
out, hidden = cell(inputs, hidden)
print("batch input size", inputs.size(), "out size", out.size())

sequence input size torch.Size([3, 5, 4]) out size torch.Size([3, 5, 2])
batch input size torch.Size([5, 3, 4]) out size torch.Size([5, 3, 2])


## Example: Teach RNN 'hihell' to 'ihello'

loss = cross entropy

In [8]:
import torch
import torch.nn as nn
from torch.autograd import Variable

torch.manual_seed(777)

idx2char = ['h', 'i', 'e', 'l', 'o']

# Teach hihell -> ihello
x_data = [0, 1, 0, 2, 3, 3] # hihell
one_hot_lookup = [[1, 0, 0, 0, 0],  # 0
                  [0, 1, 0, 0, 0],  # 1
                  [0, 0, 1, 0, 0],  # 2
                  [0, 0, 0, 1, 0],  # 3
                  [0, 0, 0, 0, 1]]  # 4

y_data = [1, 0, 2, 3, 3, 4]    # ihello
x_one_hot = [one_hot_lookup[x] for x in x_data]

# As we have one batch of samples, we will change them to variables only once
inputs = Variable(torch.Tensor(x_one_hot))
labels = Variable(torch.LongTensor(y_data))

# Parameters
num_classes = 5
input_size = 5 #one-hot size
hidden_size = 5 # output from the LSTM. 5 to directly predict one-hot
batch_size =1 # one sentence
sequence_length = 1 # One by one
num_layers = 1 # one-layer RNN

class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.rnn = nn.RNN(input_size = input_size, hidden_size = hidden_size, batch_first = True)

    def forward(self, hidden, x):
        # Reshape input (batch first)
        x = x.view(batch_size, sequence_length, input_size)

        # Propagate input through RNN
        # Input: (batch, seq_len, input_size)
        # hidden: (num_layers*num_directions, batch, hidden_size)

        out, hidden = self.rnn(x, hidden)
        return hidden, out.view(-1, num_classes)

    def init_hidden(self):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size)
        return Variable(torch.zeros(num_layers, batch_size, hidden_size))

# Instantiate RNN model
model = Model()
print(model)

# Set loss and optimizer function
# CrossENtropyLoss = LogSoftmax + NLLLoss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr =0.1)

# Train the model
for epoch in range(100):
    optimizer.zero_grad()
    loss = 0
    hidden = model.init_hidden()

    print("Predicted String: ")
    for input, label in zip(inputs, labels):
        hidden, output = model(hidden, input)
        val, idx = output.max(1)
        print(idx2char[idx.item()],end="")
        loss += criterion(output, torch.LongTensor([label]))

    print(", epoch: %d, loss: %1.3f\n" % (epoch+1, loss))

    loss.backward()
    optimizer.step()

print("Learning Finished")


Model(
  (rnn): RNN(5, 5, batch_first=True)
)
Predicted String: 
llllll, epoch: 1, loss: 10.155

Predicted String: 
llllll, epoch: 2, loss: 9.137

Predicted String: 
llllll, epoch: 3, loss: 8.355

Predicted String: 
llllll, epoch: 4, loss: 7.577

Predicted String: 
llllll, epoch: 5, loss: 6.876

Predicted String: 
lhelll, epoch: 6, loss: 6.327

Predicted String: 
ihelll, epoch: 7, loss: 6.014

Predicted String: 
ihelll, epoch: 8, loss: 5.787

Predicted String: 
ihelll, epoch: 9, loss: 5.477

Predicted String: 
ihelll, epoch: 10, loss: 5.274

Predicted String: 
ihelll, epoch: 11, loss: 5.041

Predicted String: 
ihello, epoch: 12, loss: 4.827

Predicted String: 
ihello, epoch: 13, loss: 4.676

Predicted String: 
ihello, epoch: 14, loss: 4.550

Predicted String: 
ihello, epoch: 15, loss: 4.430

Predicted String: 
ihello, epoch: 16, loss: 4.305

Predicted String: 
ihello, epoch: 17, loss: 4.164

Predicted String: 
ihelll, epoch: 18, loss: 4.003

Predicted String: 
ihelll, epoch: 19, loss: 

### RNN with a sequence

In [27]:
import torch
import torch.nn as nn
from torch.autograd import Variable

torch.manual_seed(777)  # reproducibility


idx2char = ['h', 'i', 'e', 'l', 'o']

# Teach hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

y_data = [1, 0, 2, 3, 3, 4]    # ihello

# As we have one batch of samples, we will change them to variables only once
inputs = Variable(torch.Tensor(x_one_hot))
labels = Variable(torch.LongTensor(y_data))

num_classes = 5
input_size = 5  # one-hot size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6
num_layers = 1  # one-layer rnn


class RNN(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(RNN, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length

        self.rnn = nn.RNN(input_size=5, hidden_size=5, batch_first=True)

    def forward(self, x):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size) for batch_first=True
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))

        # Reshape input
        x.view(x.size(0), self.sequence_length, self.input_size)

        # Propagate input through RNN
        # Input: (batch, seq_len, input_size)
        # h_0: (num_layers * num_directions, batch, hidden_size)

        out, _ = self.rnn(x, h_0)
        return out.view(-1, num_classes)


# Instantiate RNN model
rnn = RNN(num_classes, input_size, hidden_size, num_layers)
print(rnn)

# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.1)

# Train the model
for epoch in range(100):
    outputs = rnn(inputs)
    optimizer.zero_grad()
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    result_str = [idx2char[c] for c in idx.squeeze()]
    print("epoch: %d, loss: %1.3f" % (epoch + 1, loss.item()))
    print("Predicted string: ", ''.join(result_str))

print("Learning finished!")

RNN(
  (rnn): RNN(5, 5, batch_first=True)
)
epoch: 1, loss: 1.693
Predicted string:  llllll
epoch: 2, loss: 1.523
Predicted string:  llllll
epoch: 3, loss: 1.393
Predicted string:  llllll
epoch: 4, loss: 1.263
Predicted string:  llllll
epoch: 5, loss: 1.146
Predicted string:  llllll
epoch: 6, loss: 1.055
Predicted string:  lhelll
epoch: 7, loss: 1.002
Predicted string:  ihelll
epoch: 8, loss: 0.965
Predicted string:  ihelll
epoch: 9, loss: 0.913
Predicted string:  ihelll
epoch: 10, loss: 0.879
Predicted string:  ihelll
epoch: 11, loss: 0.840
Predicted string:  ihelll
epoch: 12, loss: 0.805
Predicted string:  ihello
epoch: 13, loss: 0.779
Predicted string:  ihello
epoch: 14, loss: 0.758
Predicted string:  ihello
epoch: 15, loss: 0.738
Predicted string:  ihello
epoch: 16, loss: 0.717
Predicted string:  ihello
epoch: 17, loss: 0.694
Predicted string:  ihello
epoch: 18, loss: 0.667
Predicted string:  ihelll
epoch: 19, loss: 0.643
Predicted string:  ihelll
epoch: 20, loss: 0.647
Predicted s

### Combine RNN + Linear using embedding

In [28]:
import torch
import torch.nn as nn
from torch.autograd import Variable

torch.manual_seed(777)  # reproducibility


idx2char = ['h', 'i', 'e', 'l', 'o']

# Teach hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
y_data = [1, 0, 2, 3, 3, 4]    # ihello

# As we have one batch of samples, we will change them to variables only once
inputs = Variable(torch.LongTensor(x_data))
labels = Variable(torch.LongTensor(y_data))

num_classes = 5
input_size = 5
embedding_size = 10  # embedding size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6
num_layers = 1  # one-layer rnn


class Model(nn.Module):

    def __init__(self, num_layers, hidden_size):
        super(Model, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(input_size=embedding_size,
                          hidden_size=5, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size)
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))

        emb = self.embedding(x)
        emb = emb.view(batch_size, sequence_length, -1)

        # Propagate embedding through RNN
        # Input: (batch, seq_len, embedding_size)
        # h_0: (num_layers * num_directions, batch, hidden_size)
        out, _ = self.rnn(emb, h_0)
        return self.fc(out.view(-1, num_classes))


# Instantiate RNN model
model = Model(num_layers, hidden_size)
print(model)

# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# Train the model
for epoch in range(100):
    outputs = model(inputs)
    optimizer.zero_grad()
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    result_str = [idx2char[c] for c in idx.squeeze()]
    print("epoch: %d, loss: %1.3f" % (epoch + 1, loss.item()))
    print("Predicted string: ", ''.join(result_str))

print("Learning finished!")

Model(
  (embedding): Embedding(5, 10)
  (rnn): RNN(10, 5, batch_first=True)
  (fc): Linear(in_features=5, out_features=5, bias=True)
)
epoch: 1, loss: 1.768
Predicted string:  eheohh
epoch: 2, loss: 1.396
Predicted string:  oheiii
epoch: 3, loss: 1.132
Predicted string:  iheloo
epoch: 4, loss: 0.949
Predicted string:  ihello
epoch: 5, loss: 0.798
Predicted string:  ihelll
epoch: 6, loss: 0.659
Predicted string:  ihelll
epoch: 7, loss: 0.547
Predicted string:  ihelll
epoch: 8, loss: 0.449
Predicted string:  ihelll
epoch: 9, loss: 0.384
Predicted string:  ihelll
epoch: 10, loss: 0.343
Predicted string:  ihello
epoch: 11, loss: 0.304
Predicted string:  ihello
epoch: 12, loss: 0.235
Predicted string:  ihello
epoch: 13, loss: 0.227
Predicted string:  ihello
epoch: 14, loss: 0.239
Predicted string:  iheloo
epoch: 15, loss: 0.233
Predicted string:  iheloo
epoch: 16, loss: 0.147
Predicted string:  ihello
epoch: 17, loss: 0.321
Predicted string:  ihelll
epoch: 18, loss: 0.139
Predicted string: